In [ ]:
from Training.SNN.labelHelper import LabelHelper
from Training.SNN.patchHelper import PatchHelper
from Training.SNN.patchLoader import PatchLoader
from shapely import geometry as SH
import random
import config

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.image as pltimage
import threading
from PIL import Image

from Training.SNN.model import Network, NetworkResnet, ContrastiveLoss, DataLoaderDynamic, DataLoaderClustering
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
import torchvision.utils
import numpy as np
import random
from PIL import Image
import torch
from torch.autograd import Variable
import PIL.ImageOps    
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [ ]:
# evalulate against a evalutation dataset
config.ANNOTATION_FILE = "./labels/val_cluster.xml"
data = []

if config.DEVICE == "cuda":
    cnn = NetworkResnet().cuda()
    test = torch.load(config.MODEL,map_location=torch.device('cuda'))
else:
    cnn = NetworkResnet()
    test = torch.load(config.MODEL,map_location=torch.device('cpu'))

print(cnn.load_state_dict(test))
cnn.eval()




In [ ]:


labelHelper = LabelHelper(config.ANNOTATION_FILE, resize=0.1)

patchLoader = PatchLoader(labelHelper, superPixelSize= [25, 25], patchSize=6)
count = 0
for i in patchLoader:
      data = data + i['combinations']
      #showImageWithLabelAndSP(i, patchLoader, 200)



In [ ]:

convert_tensor = transforms.Compose(
            [transforms.ToTensor(),
             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


test_dataset = DataLoaderDynamic(data, convert_tensor)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, pin_memory=True, num_workers=4)

print(len(data))

In [ ]:
import uuid
def show_image(i1, i2, label, e, save=False):
    fig = plt.figure(figsize=(2, 1), dpi=300)
    plt.axis('off')
    plt.title("Label:" + str(label.item()) + " Guess: " + str(e.item())[:5])
    fig.add_subplot(1,2, 1)
    plt.axis('off')
    plt.imshow(i1[0].permute(1,2,0).numpy()) 
    fig.add_subplot(1,2, 2)
    plt.axis('off')
    plt.imshow(i2[0].permute(1,2,0).numpy())
    if save:
        fig.savefig('./incorrect/' + str(uuid.uuid4()) + ".png",dpi=300, bbox_inches = "tight")
    else: 
        plt.show()

checkImages = []

total = 0
correcta = 0
correctb = 0
incorrecta = 0
incorrectb = 0

t_correct = 0
t_total = 0

same_label  = []
diff_label = []
lastfile = ("",)

margin = 2

output_2 = {}

for i1, ii1, i2, ii2, label, filename in test_loader:
    t_total = t_total + 1
    total = total + 1
    if config.DEVICE == "cuda":
        i1 = i1.cuda()
        i2 = i2.cuda()
        ii1 = ii1.cuda()
        ii2 = ii2.cuda()
        label = label.cuda()
    output=cnn(i1, ii1, i2, ii2)
    #print(output)
    a=output[0]-output[1]
    d=torch.sum(torch.pow(a,2),1)
    e=torch.sqrt(d).cpu()

    if not filename[0] in output_2:
      output_2[filename[0]] = []

    output_2[filename[0]].append({'label': label.item(), 'value': e.item()})
    
    
    if label.item() == 0:
        same_label.append(e.item())
    else:
        diff_label.append(e.item())

    if label.item()==0 and e.item()<margin:
        #show_image(i1.cpu(), i2.cpu(), label, e)
        t_correct += 1
        correcta += 1
    elif label.item()!=0 and e.item()>=margin:
        t_correct += 1
        correctb += 1
    elif label.item()==0 and e.item()>=margin:
        if filename[0] in checkImages:
            show_image(i1.cpu(), i2.cpu(), label, e)
        incorrecta += 1
    else:        
        if filename[0] in checkImages:
            show_image(i1.cpu(), i2.cpu(), label, e)
        incorrectb += 1

    if lastfile != filename:
        print(lastfile[0] + " Total: " + str(total) + " Correct A: " + str(correcta) + " Correct B: " + str(correctb) + " Incorrect A: " + str(incorrecta) + " Incorrect B: " + str(incorrectb) + " Percentage: " + str(((correcta + correctb)/total)*100) + "%")
        total = 0
        correcta = 0
        correctb = 0
        incorrecta = 0
        incorrectb = 0
        lastfile = filename


print(lastfile[0] + " Total: " + str(total) + " Correct A: " + str(correcta) + " Correct B: " + str(correctb) + " Incorrect A: " + str(incorrecta) + " Incorrect B: " + str(incorrectb) + " Percentage: " + str(((correcta + correctb)/total)*100) + "%")


In [ ]:
from statistics import mean, median

print(median(diff_label))
print(mean(diff_label))
print(min(diff_label), max(diff_label))
print(median(same_label))
print(mean(same_label))
print(min(same_label), max(same_label))

print(len(diff_label))
print(len(same_label))

print(t_total, t_correct, (t_correct/t_total) * 100)

In [ ]:
# export results for later use in evaluate_SNN.ipynb

with open('./results/pairwise.txt', 'x') as f:
    f.write(str(output_2))

In [ ]:
fig = plt.figure(figsize =(5, 4))


ax = fig.add_axes([0, 0, 1, 1])
# Creating plot
ax.boxplot([diff_label, same_label])
 
 
# show plot
plt.show()